<a href="https://colab.research.google.com/github/kmk4444/Autonomic_agents/blob/main/Part3_openai_assistant_structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- OpenAI created Assistant because of benefit of agents. There are four parts which are Assistant, Thread, Message and Run.
- Assistant means like agent. It is consisted of Name, Instructions and Tools.
- Thanks to thread, we run assistant.
- Then, assistant create message like thread_01.
- Lastly, it added to the end of the chain.

Requirements.txt

In [ ]:
!touch requirements.txt
!echo langchain >> requirements.txt
!echo langchain-openai >> requirements.txt
!echo langchain-google-genai >> requirements.txt
!echo langchainhub >> requirements.txt
!echo python-dotenv >> requirements.txt
!echo streamlit >> requirements.txt
!echo duckduckgo-search >> requirements.txt
!echo anthropic >> requirements.txt
!echo beautifulsoup4 >> requirements.txt
!echo autogenstudio >> requirements.txt
!echo crewai >> requirements.txt

terminal / bash komutu

In [ ]:
pip install -r requirements.txt

In [ ]:
%%writefile assistant_helper.py

from openai import OpenAI
import time
import os
from dotenv import load_dotenv

#load_dotenv()

#my_key_openai = os.getenv("openai_apikey")
my_key_openai="---"

client = OpenAI(api_key=my_key_openai)

assistant_id = "asst_9PkxJouK0097L2lrUxSznB2U" #| Python Code Assistant

#create a new thread
#create a new run
#create a new message
#take the AI response

def start_net_thread():
  thread = client.beta.threads.create()
  thread_id = thread.id

  return thread_id

def add_message_to_thread(thread_id,prompt):
  message = client.beta.threads.messages.create(
      thread_id=thread_id,
      role="user",
      content=prompt,
  )

def execute_run_cycle(thread_id):
  run = client.beta.threads.runs.create(
      thread_id=thread_id,
      assistant_id=assistant_id
  )

  while True:

    run = client.beta.threads.runs.retrieve(
        thread_id=thread_id,
        run_id=run.id
    )

    if run.completed_at:
      elapsed = run.completed_at - run.created_at
      lapsed = time.strftime("%H:%M:%S", time.gmtime(elapsed))
      print(f"Run completed in {elapsed}")
      print("-"*100)
      break
    time.sleep(1)

  messages = client.beta.threads.messages.list(thread_id=thread_id)
  last_message = messsages.data[0]

  AI_Response = last_message.content[0].text.value

  return AI_Response


In [ ]:
%%writefile app_assistant.py

import streamlit as st
import assistant_helper

if "messages" not in st.session_state:
  st.session_state.messages = []
  st.session_state.thread_id= None

st.set_page_config(page_title="Python Kodlama Asistanı", page_icon=":speech_balloon:")
st.title("Python Kodlama Asistanı")
st.divider()

for message in st.session_state.messages:
  with st.chat_message(message["role"]):
    st.markdown(message["content"])

if prompt := st.chat_input("Mesajınızı yazınız"):
  with st.chat_message("user"):
    st.markdown(prompt)

  with st.spinner("Yanıt oluşturuluyor...")
    if st.session_state.thread_id is None:
      st.session_state.thread_id = assistant_helper.start_new_thread()

    assistant_helper.add_message_to_thread(thread_id=st.session_state.thread_id, prompt=prompt)

    AI_Response = assistant_helper.execute_run_cycle(thread_id=st.session_state.thread_id)

    with st.chat_message("assistant"):
      st.markdown(AI_Response)

   st.session_state_messages.append({"role":"user","content":prompt})
   st.session_state.messages.append({"role":"assistant","content":AI_Response})